## This notebook shows how to delete an existing slice

### Configure the environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

### Create slice manager object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host,refresh_token=fabric_refresh_token, project_name='all', scope='all')

### Refresh the ID Token

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print("New Refresh Token: {}".format(fabric_refresh_token))
print("Stored new Refresh Token")
%store fabric_refresh_token

### Create a basic slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='RENC')

# Set capacities
cap = Capacities()
cap.set_fields(core=4, ram=64, disk=500)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n1.add_component(ctype=ComponentType.SmartNIC, model='ConnectX-5', name='nic1')


# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='ToBeRemoved', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))

### Query existing Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

### Set variable for the Slice ID from Slice 0

In [ ]:
# Set the Slice ID from output of the above command
slice_id=slices[0].slice_id

### Delete Slice 0

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Response received {}".format(result))

### Query existing Slices after Delete

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))